
---

# Tema: Análisis de Mercado de Ropa de Tallas Extra. Parte 2

---





## Retomando el tema de la Parte 1 (Anterior Módulo)
En la primera etapa del módulo, nuestro proyecto tomó como estudio una problemática creciente en México, la obesidad y el sobrepeso.

Nos centramos en la vida cotidiana de  este sector, específicamente en el ámbito de la industria de la confección , analizando la escasa  oferta y comercialización  de sucursales enfocadas a la venta de Ropa de Talla Extra que son cada vez más solicitadas en personas desde  temprana edad y de manera constante en la edad adulta.

En la primera parte del proyecto se obtuvo mediante el análisis de 4 variables un top 5 de estados candidatos para introducir tiendas especializadas en la venta de ropa de tallas extra, seleccionando al estado de Nuevo León como la mejor opción.



---

## Análisis y manipulación de datos
En este Notebook nos centraremos en la limpieza y el merge de distintas bases de datos para obtener un dataframe final que nos servirá para graficar, visualizar los datos y utilizarla en la Notebook posterior, así como utilizar los datos para la siguiente fase de Machine Learning e Inteligencia Artificial en búsqueda de la mejor población compuesta por los puntos de interés (tiendas) a través de la implementación de un Algoritmo Genético.

##1. Objetivos:
*   Relacionar datasets de tiendas dedicadas a la venta de ropa en la entidad federativa de Nuevo León, agregando información sobre otras variables como porcentaje de obesidad, ventas, datos de Google Maps, concentración de tiendas.
*   Elegir un pool de tiendas candidatas como canales de distribución de ropa de tallas extra, así como de lugares estratégicos para la difusión de la publicidad de una tienda en línea de venta de ropa de tallas extra.

Tal como se mencionó, en esta parte del proyecto no solo nos centramos a la localización de tiendas físicas, sino abarcar el rubro de redes de dirtribución de nuestro producto, así como la introducción de una tienda de venta on line.

---

##2. Desarrollo:
*   Analizar y limpiar la base de datos adquirida de Ventas-BasedeDatos.com
*   Analizar y limpiar la base de datos de Empresas dedicadas a la venta de Ropa obtenida a través del API de DENUE.
*   Realizar merge entre las dos bases de datos anteriores a través de un campo en común, en este caso el Nombre de los Municipios, con ayuda de otra base datos de Codigos Postales y Nombres de Municipios.
*   Realizar merge con la base de datos del Porcentaje de Obesidad por Municipio.
*   Realizar merge con la base de datos de la cantidad de ventas de ropa.
*   Calcular la concentración de tiendas por Municipio y por Colonia.
*   Del dataframe final, seleccionar el listado de tiendas potenciales que podrían ser candidatas.
*   Consultar el API de LivePopularTime que hace un scraping de datos sobre Google maps en cada una de las tiendas del dataframe final, accediendo a datos como rating, horarios, tiempo promedio de estancia, número de comentarios y placeid.
*   Abordar una propuesta de los planes futuros para los siguientes módulos


---
## Bases de Datos utilizadas
*   API del DENUE, búsqueda de establecimientos dedicados a la venta de ropa.
  *   Descripción: La API del DENUE permite consultar datos de identificación, ubicación, actividad económica y tamaño de más de 5 millones de establecimientos a nivel nacional, por entidad federativa y municipio. La búsqueda se hizo sobre la entidad Federativa de Nuevo León. Esta base de datos se obtuvo del anterior proyecto de R, pero se retoman los datos.
  *  URL: https://www.inegi.org.mx/servicios/api_denue.html#:~:text=La%20API%20del%20DENUE%20te,por%20entidad%20federativa%20y%20municipio
*   Base de datos Adquirida Venta-BasesdeDatos.com:
  *   Descripción: Base de datos actualizada sobre empresas dedicadas a la venta de ropa en la entidad federativa de Nuevo León, incluyendo el tamaño de cada empresa.
  *   URL:https://venta-basesdedatos.com/directorios-correos-empresas/?wpf_count=100&filter_cat_0=1171
*   Base de datos de la Encuesta Nacional de Salud y Nutrición (ENSANUT) 2018
  *   Descripción: Porcentaje de Obesidad por Entidad Federativa y Municipios en México (2018).
  *   URL: https://www.inegi.org.mx/programas/ensanut/2018/#Tabulados

*  Comercio al por Menor de Ropa, Bisuteria y Accesorios de Vestir (2020):
  *   URL: https://datamexico.org/es/profile/industry/comercio-al-por-menor-de-ropa-bisuteria-y-accesorios-de-vestir



---
## Base de datos adquirida Venta-BasedeDatos.com de Nuevo León
---

Se decidió adquirir una bade de datos debido a que representa una gran ayuda para el análisis, ya que cuenta con datos actualizados de tiendas de ropa en el estado de Nuevo Léon. 

In [ ]:
import pandas as pd
import re
import json
import requests
import certifi

In [ ]:
nuevo_leon = pd.read_csv('https://raw.githubusercontent.com/Axelflg/datasets/main/comprada.csv',encoding='ISO-8859-1')

### 1. Analizamos los datos 
Esta base de datos está compuesta por campos como Nombre Empresa, Estado, Municipio, Teléfono, Actividad Empresarial y Tamaño de la Empresa. Este último campo es especialmente importante ya que nos permite diferenciar si una empresa es A: Grande empresa, B: Mediana empresa o C: Pequeña empresa. 
Dicho campo nos permite considerar con mayor importancia a las empresas de tamaño A y B debido a que son tiendas que se pueden convertir en nuestros distribuidores potenciales. No obstante, también se considerarán en este estudio las empresas de Tamaño C ya que abarcan un sector importante de ventas en el estado de Nuevo León.

In [ ]:
# Obtenemos información de los campos
nuevo_leon.dtypes

In [ ]:
nuevo_leon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 5 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Nombre de Empresa / Razón Social     1471 non-null   object
 1   Municipio / Delegación               1471 non-null   object
 2   Teléfono                             1471 non-null   int64 
 3   Actividad Empresarial o Profesional  1471 non-null   object
 4   Tamaño de Empresa                    1471 non-null   object
dtypes: int64(1), object(4)
memory usage: 57.6+ KB


### 2. Limpiamos los datos

Renombramos campos y eliminamos lo que no nos sirve

In [ ]:
# Renombramos los campo del DataFrame nuevo_leon con nombres más entendibles
column_name_nuevo_leon = {
    'Nombre de Empresa / Razón Social': 'nombre_tienda',
    'Municipio / Delegación': 'municipio',
    'Teléfono': 'telefono',
    'Actividad Empresarial o Profesional': 'giro',
    'Tamaño de Empresa': 'dimension',
}
nuevo_leon_format = nuevo_leon.rename(columns=column_name_nuevo_leon)

In [ ]:
# Eliminamos la columna giro y teléfono que no nos sirven
nuevo_leon_format = nuevo_leon_format.drop(columns=['giro', 'telefono'])

In [ ]:
# Función para eliminar números de una cadena de texto
def sin_numero(value):
  return re.sub('\d', '', value)

In [ ]:
# Eliminamos números en el nombre_tienda del dataframe nuevo_leon_format
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].map(sin_numero)

In [ ]:
# Eliminamos espacios al incio y final
nuevo_leon_format['nombre_tienda']  = nuevo_leon_format['nombre_tienda'] .str.strip()
nuevo_leon_format['municipio']  = nuevo_leon_format['municipio'] .str.strip()

In [ ]:
# Visualizamos como quedaron nuestros datos
nuevo_leon_format.head()

,nombre_tienda,municipio,dimension
0,Boutique wapa,Santa Catarina,C
1,Accesorios lentes moños,General Escobedo,C
2,Shek unique women,San Nicolas De Los Garza,C
3,Ted baker,San Pedro Garza Garcia,C
4,"Accesorios r s,tienda de ropa",San Pedro Garza Garcia,C


Se modifican los nombres de las tiendas de tal forma que se quitan las letras mayúsculas con excepción de la primera; así como se eliminan todos los acentos. Esto se hace para que los nombres coincidan entre diferentes dataframes para un merge posterior


In [ ]:
# Aquí está el filtro para los nombres de las tiendas
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].apply(lambda x: x.capitalize())
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].str.replace('á','a')
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].str.replace('é','e')
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].str.replace('í','i')
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].str.replace('ó','o')
nuevo_leon_format['nombre_tienda'] = nuevo_leon_format['nombre_tienda'].str.replace('ú','u')

In [ ]:
# Aquí está el filtro para los nombres de municipio
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.replace('á','a')
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.replace('é','e')
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.replace('í','i')
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.replace('ó','o')
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.replace('ú','u')

In [ ]:
# Ponemos en formato de título
nuevo_leon_format['municipio'] = nuevo_leon_format['municipio'].str.title()

In [ ]:
# Hay registros repetidos, los eliminamos basado en todas las columnas
nuevo_leon_format2 = nuevo_leon_format.drop_duplicates(keep='first')

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_format2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1359 entries, 0 to 1470
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nombre_tienda  1359 non-null   object
 1   municipio      1359 non-null   object
 2   dimension      1359 non-null   object
dtypes: object(3)
memory usage: 42.5+ KB


In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_format2.head()

,nombre_tienda,municipio,dimension
0,Boutique wapa,Santa Catarina,C
1,Accesorios lentes moños,General Escobedo,C
2,Shek unique women,San Nicolas De Los Garza,C
3,Ted baker,San Pedro Garza Garcia,C
4,"Accesorios r s,tienda de ropa",San Pedro Garza Garcia,C


---
## Base de datos Códigos Postales de Nuevo León
---

###1. Leemos la base de datos y analizamos los datos

Esta base de datos contiene información relativa a los códigos postales específicos de la entidad federativa de Nuevo León. En nuestro caso solo nos interesan los datos relacionados con Código Postal y el Nombre del Municipio, ya que utilizaremos esta Base de Datos para hacer un Merge posterior.

In [ ]:
# Leemos BD de Códigos Posatales
# https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx
nuevo_leon_cp = pd.read_csv('https://raw.githubusercontent.com/Axelflg/datasets/main/Nuevo%20Leon%20CP.csv')

In [ ]:
nuevo_leon_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4889 entries, 0 to 4888
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   d_codigo          4889 non-null   int64  
 1   d_asenta          4889 non-null   object 
 2   d_tipo_asenta     4889 non-null   object 
 3   D_mnpio           4889 non-null   object 
 4   d_estado          4889 non-null   object 
 5   d_ciudad          3410 non-null   object 
 6   d_CP              4889 non-null   int64  
 7   c_estado          4889 non-null   int64  
 8   c_oficina         4889 non-null   int64  
 9   c_CP              0 non-null      float64
 10  c_tipo_asenta     4889 non-null   int64  
 11  c_mnpio           4889 non-null   int64  
 12  id_asenta_cpcons  4889 non-null   int64  
 13  d_zona            4889 non-null   object 
 14  c_cve_ciudad      3410 non-null   float64
dtypes: float64(2), int64(7), object(6)
memory usage: 573.1+ KB


In [ ]:
nuevo_leon_cp.head()

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,d_estado,d_ciudad,d_CP,c_estado,c_oficina,c_CP,c_tipo_asenta,c_mnpio,id_asenta_cpcons,d_zona,c_cve_ciudad
0,64000,Monterrey Centro,Colonia,Monterrey,Nuevo León,Monterrey,64008,19,64008,NaN,9,39,1,Urbano,7.0
1,64000,La Finca,Condominio,Monterrey,Nuevo León,Monterrey,64008,19,64008,NaN,10,39,3429,Urbano,7.0
2,64004,Amway,Gran usuario,Monterrey,Nuevo León,Monterrey,64008,19,64008,NaN,22,39,5,Urbano,7.0
3,64007,Secretaría de Hacienda y Crédito Público,Gran usuario,Monterrey,Nuevo León,Monterrey,64008,19,64008,NaN,22,39,8,Urbano,7.0
4,64009,Palacio de Gobierno Del Estado de Nuevo León,Gran usuario,Monterrey,Nuevo León,Monterrey,64008,19,64008,NaN,22,39,10,Urbano,7.0


### 2. Limpiamos los datos

Renombramos campos y eliminamos lo que no nos sirve.

In [ ]:
# Eliminamos las columna que no nos sirven
nuevo_leon_cp_format = nuevo_leon_cp.drop(columns=['d_asenta', 'd_tipo_asenta', 'd_estado', 'd_ciudad', 'd_CP', 'c_estado', 'c_oficina', 'c_CP', 'c_tipo_asenta', 'c_mnpio', 'id_asenta_cpcons', 'd_zona', 'c_cve_ciudad'])

In [ ]:
# Renombramos campos del DataFrame nuevo_leon_cp_format
column_name_nuevo_leon_cp = {
    'd_codigo': 'codigo_postal',
    'D_mnpio': 'municipio'
}
nuevo_leon_cp_format = nuevo_leon_cp_format.rename(columns=column_name_nuevo_leon_cp)

In [ ]:
# Eliminamos registros repetidos
nuevo_leon_cp_format2 = nuevo_leon_cp_format.drop_duplicates(keep='first')

In [ ]:
# Hacemos Index (Llave primaria) a la columna de codigo Postal
nuevo_leon_cp_format2 = nuevo_leon_cp_format2.set_index('codigo_postal', drop=True)

Es importante verificar que los nombres de los municipios tengan la misma nomenclatura en este dataframe y en el que vamos a hacer merge (Base de Datos de DENUE), debido a que el campo de 'municipio' servirá como una de las llaves para realizar la unión entre las Base de Datos DENUE y la Base de Datos Venta-BasedeDatos.com. 
Se seleccionó el nombre del municipio como llave debido a que la Base de Datos Venta-BasedeDatos.com solo cuenta con dicho campo, y la Base de Datos DENUE cuenta con el Código Postal, pero haciendo el merge también puede tener el nombre del Municipio.

In [ ]:
# Remplazamos acentos
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.replace('á','a')
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.replace('é','e')
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.replace('í','i')
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.replace('ó','o')
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.replace('ú','u')

In [ ]:
# Ponemos en formato de título
nuevo_leon_cp_format2['municipio'] = nuevo_leon_cp_format2['municipio'].str.title()

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_cp_format2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 64000 to 67996
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   municipio  1096 non-null   object
dtypes: object(1)
memory usage: 17.1+ KB


In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_cp_format2.head()

,municipio
codigo_postal,
64000,Monterrey
64004,Monterrey
64007,Monterrey
64009,Monterrey
64010,Monterrey


---
## Base de datos DENUE - Tiendas de Ropa en la Entidad Federativa de Nuevo León
---

DENUE es una API que el INEGI pone a disposición de todo el mundo, la cual permite consultar datos de identificación, ubicación, actividad económica y tamaño de más de 5 millones de establecimientos a nivel nacional, por entidad federativa y municipio. Nosotros nos centramos en la entidad federativa de Nuevo León, y debido a que en el anterior módulo ya habíamos hecho esta implementación con R, en esta ocasión solo estamos leyendo el dataset generado en su momento.

### 1. Leemos la base de datos

In [ ]:
nuevo_leon_denue = pd.read_csv('https://raw.githubusercontent.com/Axelflg/datasets/main/nuevoleon.csv',encoding='ISO-8859-1')

### 2. Limpiamos los datos

Renombramos campos y eliminamos lo que no nos sirve.

In [ ]:
# Eliminamos la columna giro que no nos sirve
nuevo_leon_denue = nuevo_leon_denue.drop(columns=['Unnamed: 0', 'Id'])

In [ ]:
# Renombramos campos del DataFrame nuevo_leon_denue
column_name_nuevo_leon_denue = {
    'Nombre': 'nombre_tienda',
    'Razon_social': 'razon_social',
    'Clase_actividad': 'clase',
    'Estrato': 'estrato',
    'Tipo_vialidad': 'tipo_vialidad',
    'Calle': 'calle',
    'Num_Exterior': 'num_exterior',
    'Num_Interior': 'num_interior',
    'Colonia': 'colonia',
    'CP': 'codigo_postal',
    'Ubicacion': 'ubicacion',
    'Telefono': 'telefono',
    'Correo_e': 'correo',
    'Sitio_internet': 'pagina_web',
    'Tipo': 'tipo',
    'Longitud': 'longitud',
    'Latitud': 'latitud',
    'tipo_corredor_industrial': 'tipo_corredor_industrial',
    'nom_corredor_industrial': 'nom_corredor_industrial',
    'numero_local': 'numero_local',
}
nuevo_leon_denue_format = nuevo_leon_denue.rename(columns=column_name_nuevo_leon_denue)

In [ ]:
# Eliminamos números en el nombre_tienda del dataframe nuevo_leon_denue_format
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].map(sin_numero)

In [ ]:
# Eliminamos espacios al incio y final
nuevo_leon_denue_format['nombre_tienda']  = nuevo_leon_denue_format['nombre_tienda'] .str.strip()

In [ ]:
# Quitamos los registros que tienen na
nuevo_leon_denue_format = nuevo_leon_denue_format[nuevo_leon_denue_format['codigo_postal'].notna()]

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_denue_format.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106


In [ ]:
# Convertimos a enteros el Código postal
nuevo_leon_denue_format['codigo_postal'] = pd.to_numeric(nuevo_leon_denue_format['codigo_postal'], downcast='signed')

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_denue_format.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106


Modificamos los nombres de las tiendas de tal forma que se quiten las letras mayúsculas con excepción de la primera, además, se eliminan todos los acentos. Esto se hace para que los nombres coincidan entre diferentes dataframes para un merge posterior.

In [ ]:
# Aquí está el filtro para los nombres de las tiendas
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].apply(lambda x: x.capitalize())
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].str.replace('á','a')
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].str.replace('é','e')
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].str.replace('í','i')
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].str.replace('ó','o')
nuevo_leon_denue_format['nombre_tienda'] = nuevo_leon_denue_format['nombre_tienda'].str.replace('ú','u')

Realizamos el merge de la Base de Datos DENUE con el Código Postal, para tener los Nombres de los Municipios de cada uno de los registros dentro de la Base de Datos de DENUE en el dataframe 'nuevo_leon_denue_format2'.
El objetivo es que 'nuevo_leon_format2' cuente con el dato de 'Municipio', ya que posteriormente servirá como una de las llaves para hacer Merge con la Base de Datos adquirida Venta-BasedeDatos.com.

In [ ]:
# Realizamos el merge entre la Base de Datos DENUE y la base de datos de Códigos Postales
nuevo_leon_denue_format2 = pd.merge(nuevo_leon_denue_format, nuevo_leon_cp_format2, left_on='codigo_postal', right_index=True).sort_index()

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_denue_format2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 4097
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             3996 non-null   object 
 1   razon_social              855 non-null    object 
 2   clase                     3996 non-null   object 
 3   estrato                   3996 non-null   object 
 4   tipo_vialidad             3996 non-null   object 
 5   calle                     3996 non-null   object 
 6   num_exterior              3228 non-null   float64
 7   num_interior              1214 non-null   float64
 8   colonia                   3996 non-null   object 
 9   codigo_postal             3996 non-null   int32  
 10  ubicacion                 3996 non-null   object 
 11  telefono                  1028 non-null   float64
 12  correo                    462 non-null    object 
 13  pagina_web                301 non-null    object 
 14  tipo    

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_denue_format2.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey


---
## Merge base de datos adquirida Venta-BasedeDatos.com y DENUE
---

En este punto realizamos el merge entre ambas Bases de Datos la DENUE y la adquirida Venta-BasedeDatos.com, el objetivo de esto es utilizar la Base de Datos DENUE como la principal, pero recuperar un dato muy importante de la base de datos adquirida el cual es el 'Tamaños de la empresa', este campo es un diccionario que contiene los siguientes valores:
*   A: Grande empresa
*   B: Mediana empresa
*   C: Pequeña empresa

Este dato es de gran importancia, ya que nos permite identificar fácilmente a tiendas con prestigio y bien implantadas en el mercado mexicano que se pueden convertir en competencia directa o en buenos canales de distribución de nuestro producto.

In [ ]:
# Realizamos el merge de la Base de Datos DENUE con la Base de Datos adquirida
nuevo_leon_merged = (pd.merge(nuevo_leon_denue_format2, nuevo_leon_format2, how="left", on=["nombre_tienda","municipio"]))

In [ ]:
# Hay registros repetidos, los eliminamos basado en todas las columnas
nuevo_leon_merged = nuevo_leon_merged.drop_duplicates(keep='first')

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_merged

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 4011
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             3993 non-null   object 
 1   razon_social              868 non-null    object 
 2   clase                     3993 non-null   object 
 3   estrato                   3993 non-null   object 
 4   tipo_vialidad             3993 non-null   object 
 5   calle                     3993 non-null   object 
 6   num_exterior              3237 non-null   float64
 7   num_interior              1215 non-null   float64
 8   colonia                   3993 non-null   object 
 9   codigo_postal             3993 non-null   int32  
 10  ubicacion                 3993 non-null   object 
 11  telefono                  1034 non-null   float64
 12  correo                    475 non-null    object 
 13  pagina_web                311 non-null    object 
 14  tipo    

Podemos ver claramente la predominancia de las empresas dedicadas a la venta de ropa de tamaño pequeño, sin embargo, no debemos de ignorar la importancia de las tiendas de tamaño grande y mediano.

In [ ]:
# Verificamos cuantos registros hay por Tamaño de la Empresa (A=Grande, B=Mediana, C=Pequeña)
nuevo_leon_merged.groupby('dimension').size()

dimension
A      41
B      48
C    1002
dtype: int64

---
## Merge base de datos Porcentaje de Obecidad por Municipio en Nuevo León
---


Con el fin de complementar nuestro Dataframe, utilizamos la base de 
datos de la Encuesta Nacional de Salud y Nutrición (ENSANUT) 2018 que nos permite obtener los Porcentaje de Obesidad por municipio de las entidades federativas de México 2018. Esta variable forma un punto crucial, debido a que nos indica la concentración de la población destino a la que va dirigido el producto.



### 1. Leemos la base de datos y analizamos los datos

In [ ]:
# Leedmos el dataset 
porcentaje_obesidad = pd.read_csv('https://raw.githubusercontent.com/Axelflg/datasets/main/obesidadMunicipio.csv',encoding='ISO-8859-1')

In [ ]:
porcentaje_obesidad.head()

,Unnamed: 0,UUID.Municipio,Clave.Estado,entidad_federativa,Clave.Municipio,municipio,Estimador,porcentaje_obesidad
0,1943,26054,26,Sonora,54,San Javier,Valor,79.5410
1,38,5006,5,Coahuila de Zaragoza,6,Castaños,Valor,66.9743
2,1910,26021,26,Sonora,21,La Colorada,Valor,65.6808
3,1911,26022,26,Sonora,22,Cucurpe,Valor,60.8561
4,1933,26044,26,Sonora,44,Onavas,Valor,58.6237


### 2. Limpiamos los datos
Renombramos campos y eliminamos lo que no nos sirve

In [ ]:
# Renombramos campos del DataFrame 
column_name_porcentaje_obesidad = {
    'Entidad.federativa': 'entidad_federativa',
    'Nombre.Municipio': 'municipio',
    'Porcentaje.obesidad': 'porcentaje_obesidad'
}
porcentaje_obesidad = porcentaje_obesidad.rename(columns=column_name_porcentaje_obesidad)

In [ ]:
# Filtramos solo registros del estado de Nuevo León 
nuevo_leon_obesidad = porcentaje_obesidad[porcentaje_obesidad.entidad_federativa == 'Nuevo León']

In [ ]:
# Eliminamos las columnas que no nos sirven del Dataframe del Porcentaje de Obesidad por municipio en Nuevo León
nuevo_leon_obesidad = nuevo_leon_obesidad.drop(columns=['Unnamed: 0', 'UUID.Municipio', 'Clave.Estado', 'entidad_federativa', 'Clave.Municipio', 'Estimador'])

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_obesidad.head()

,municipio,porcentaje_obesidad
76,Allende,48.7273
89,Garcia,48.2803
91,Vallecillo,48.2551
125,Cadereyta Jimenez,46.9489
163,Cienega De Flores,45.9532


In [ ]:
# Remplazamos acentos
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.replace('á','a')
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.replace('é','e')
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.replace('í','i')
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.replace('ó','o')
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.replace('ú','u')

In [ ]:
# Ponemos en formato de título para no tener problemas posteriormente con el Merge
nuevo_leon_obesidad['municipio'] = nuevo_leon_obesidad['municipio'].str.title()

In [ ]:
# Eliminamos registros repetidos
nuevo_leon_obesidad = nuevo_leon_obesidad.drop_duplicates(keep='first')

In [ ]:
# Realizamos el merge de la Base de Datos del Porcentaje de obesidad por municipio en Nuevo León, con la general que ya tenemos
# De esta forma agregamos la columna "porcentaje_obsesidad" en cada uno de los registros de tienda
nuevo_leon_merged_obs = (pd.merge(nuevo_leon_merged, nuevo_leon_obesidad, how="left", on=["municipio"]))

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_merged_obs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 3992
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             3993 non-null   object 
 1   razon_social              868 non-null    object 
 2   clase                     3993 non-null   object 
 3   estrato                   3993 non-null   object 
 4   tipo_vialidad             3993 non-null   object 
 5   calle                     3993 non-null   object 
 6   num_exterior              3237 non-null   float64
 7   num_interior              1215 non-null   float64
 8   colonia                   3993 non-null   object 
 9   codigo_postal             3993 non-null   int32  
 10  ubicacion                 3993 non-null   object 
 11  telefono                  1034 non-null   float64
 12  correo                    475 non-null    object 
 13  pagina_web                311 non-null    object 
 14  tipo    

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_merged_obs.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey,NaN,42.2707
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey,NaN,42.2707
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707


---
# Merge base de datos Porcentaje de Ventas por Municipio en Nuevo León
---

Ahora, agregamos un dato adicional a nuestro Dataframe de la base de datos Comercio al por Menor de Ropa, Bisuteria y Accesorios de Vestir (2020) que nos indica la cantidad de ventas en millones de pesos por municipio en Nuevo León, esto nos permite identificar cuáles son los municipios que mayormente se dedican a este sector.

### 1. Leemos la base de datos y analizamos los datos

In [ ]:
# Leedmos el dataset 
nuevo_leon_ventas = pd.read_csv('https://raw.githubusercontent.com/Axelflg/datasets/main/Balance-comercial-neto-2019.csv')

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_ventas.head()

,municipio,valor_comercial
0,Allende,69.939439
1,Apodaca,11257.508571
2,Cadereyta Jimenez,176.262942
3,El Carmen,28.702938
4,Santiago,35.801389


###2. Limpiamos los datos
Renombramos campos y eliminamos lo que no nos sirve

In [ ]:
# Eliminamos las columnas que no nos sirven del Dataframe del Porcentaje de Obesidad por municipio en Nuevo León
nuevo_leon_ventas = nuevo_leon_ventas.drop(columns=['Flow ID', 'Flow', 'Nation ID', 'Nation', 'State ID', 'State', 'Municipality ID', 'Trade Value Exports', 'Trade Value Delta', 'Trade Value Imports'])

In [ ]:
# Renombramos campos del DataFrame 
column_name_ventas = {
    'Municipality': 'municipio',
    'Trade Value': 'valor_comercial'
}
nuevo_leon_ventas = nuevo_leon_ventas.rename(columns=column_name_ventas)

Estandarizamos el Nombre del municipio quitando acentos y poniéndolo en formato de título con el fin de hacer un merge posterior con el dataframe principal

In [ ]:
# Remplazamos acentos
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.replace('á','a')
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.replace('é','e')
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.replace('í','i')
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.replace('ó','o')
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.replace('ú','u')

In [ ]:
# Ponemos en formato de título para no tener problemas posteriormente con el Merge
nuevo_leon_ventas['municipio'] = nuevo_leon_ventas['municipio'].str.title()

In [ ]:
# Eliminamos registros repetidos
nuevo_leon_ventas = nuevo_leon_ventas.drop_duplicates(keep='first')

In [ ]:
# Dividimo entre 1000000 para manejar mejor los datos y lo consideramos como montos en Millones de Pesos
nuevo_leon_ventas['valor_comercial'] = nuevo_leon_ventas['valor_comercial']/1000000

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_ventas.head()

,municipio,valor_comercial
0,Allende,69.939439
1,Apodaca,11257.508571
2,Cadereyta Jimenez,176.262942
3,El Carmen,28.702938
4,Santiago,35.801389


Realizamos el merge de la Base de Datos de Comercio al por Menor de Ropa, Bisuteria y Accesorios de Vestir (2020) por municipio en Nuevo León, con el dataframe principal 'nuevo_leon_merged_obs' que ya tiene el porcentaje de obesidad.
De esta forma agregamos la columna "valor_comercial" en cada uno de los registros de las tiendas.

In [ ]:
# Realizamos el merge
nuevo_leon_merged_obs_ven = (pd.merge(nuevo_leon_merged_obs, nuevo_leon_ventas, how="left", on=["municipio"]))

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_merged_obs_ven.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 3992
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             3993 non-null   object 
 1   razon_social              868 non-null    object 
 2   clase                     3993 non-null   object 
 3   estrato                   3993 non-null   object 
 4   tipo_vialidad             3993 non-null   object 
 5   calle                     3993 non-null   object 
 6   num_exterior              3237 non-null   float64
 7   num_interior              1215 non-null   float64
 8   colonia                   3993 non-null   object 
 9   codigo_postal             3993 non-null   int32  
 10  ubicacion                 3993 non-null   object 
 11  telefono                  1034 non-null   float64
 12  correo                    475 non-null    object 
 13  pagina_web                311 non-null    object 
 14  tipo    

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_merged_obs_ven.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey,NaN,42.2707,6098.849922
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey,NaN,42.2707,6098.849922
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922


**Cálculo de concentración del número de tiendas por Municipio**

Cálculamos la cantidad de Tiendas por Municipio y asignamos dichos datos como un nuevo campos a cada uno de los registros

In [ ]:
conteo_municipio = pd.DataFrame(nuevo_leon_merged.groupby(['municipio'])['nombre_tienda'].count())

In [ ]:
# Visualizamos el dataframe generado
conteo_municipio.head()

,numero_tiendas_municipio
municipio,
Abasolo,1
Allende,63
Anahuac,18
Apodaca,234
Aramberri,10


In [ ]:
# Renombramos campos del DataFrame conteo_df
column_name_conteo_municipio = {
    'nombre_tienda': 'numero_tiendas_municipio',
}
conteo_municipio = conteo_municipio.rename(columns=column_name_conteo_municipio)

In [ ]:
# Realizamos el merge de la Base de Datos con el numero de tiendas por municipio
nuevo_leon_calculo = pd.merge(nuevo_leon_merged_obs_ven, conteo_municipio, left_on='municipio', right_index=True).sort_index()

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_calculo.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial,numero_tiendas_municipio,numero_tiendas_colonia
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey,NaN,42.2707,6098.849922,1578,5
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey,NaN,42.2707,6098.849922,1578,12
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777,247,4
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922,1578,282
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922,1578,282


In [ ]:
# Calculamos la media, mediana y desviación estandar para saber que puntos tomar
nuevo_leon_calculo['numero_tiendas_municipio'].agg(['mean', 'median', 'std'])

mean      746.726521
median    313.000000
std       679.886895
Name: numero_tiendas_municipio, dtype: float64

**Cálculo de concentración de número de tiendas por colonia**

Cálculamos la cantidad de Tiendas por Colonia y asignamos dichos datos como un nuevo campos a cada uno de los registros

In [ ]:
conteo_colonia = pd.DataFrame(nuevo_leon_merged.groupby(['colonia'])['nombre_tienda'].count())

In [ ]:
# Visualizamos el dataframe generado
conteo_colonia.head()

,numero_tiendas_colonia
colonia,
1 DE JUNIO (FOMERREY 167),1
13 DE JUNIO,1
13 DE MAYO,2
15 DE SEPTIEMBRE,1
18 DE FEBRERO,2


In [ ]:
# Renombramos campos del DataFrame conteo_df
column_name_conteo_colonia = {
    'nombre_tienda': 'numero_tiendas_colonia',
}
conteo_colonia = conteo_colonia.rename(columns=column_name_conteo_colonia)

In [ ]:
# Realizamos el merge de la Base de Datos con el numero de tiendas por municipio
nuevo_leon_calculo = pd.merge(nuevo_leon_calculo, conteo_colonia, left_on='colonia', right_index=True).sort_index()

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_calculo.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial,numero_tiendas_municipio,numero_tiendas_colonia
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey,NaN,42.2707,6098.849922,1578,5
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey,NaN,42.2707,6098.849922,1578,12
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777,247,4
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922,1578,282
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922,1578,282


Calculamos la media, mediana y desviación estándar de las 4 variables que recién agregamos al dataframe, es decir:
*   Porcentaje de obesidad por Municipio
*   Número de ventas en millones de pesos por Municipio
*   Concentración de tiendas por Municipio
*   Concentración de tiendas por colonias

Dichas variables nos servirán más adelante para elegir las mejores tiendas de acuerdo a estas características


In [ ]:
# Calculamos la media, mediana y desviación estandar para saber que puntos tomar
nuevo_leon_calculo['numero_tiendas_colonia'].agg(['mean', 'median', 'std'])

mean       77.247934
median     10.000000
std       110.257687
Name: numero_tiendas_colonia, dtype: float64

In [ ]:
# Calculamos la media, mediana y desviación estandar del Porcentaje de Obesidad para saber que puntos tomar
nuevo_leon_calculo['porcentaje_obesidad'].agg(['mean', 'median', 'std'])

mean      42.169997
median    42.270700
std        2.861220
Name: porcentaje_obesidad, dtype: float64

In [ ]:
# Calculamos la media, mediana y desviación estandar del Valor Comercial para saber que puntos tomar
nuevo_leon_calculo['valor_comercial'].agg(['mean', 'median', 'std'])

mean      4612.015643
median    4634.679777
std       2765.167297
Name: valor_comercial, dtype: float64


---
## Selección de las tiendas potenciales
---


###1. Leemos la base de datos y analizamos los datos

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_calculo.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial,numero_tiendas_municipio,numero_tiendas_colonia
0,"Monterrey, plaza cumbres",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,HACIENDA DE PEÑUELAS,6771.0,NaN,CUMBRES LAS PALMAS,64349,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.397801,25.733109,CENTRO Y PLAZA COMERCIAL,PLAZA CUMBRES,1151 A 1157,Monterrey,NaN,42.2707,6098.849922,1578,5
1,"Monterrey, gal.valle ote",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",6 a 10 personas,CALLE,PASEO NARANJO,1000.0,NaN,VALLE DEL MIRADOR,64750,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.SHASA.COM,Fijo,-100.313720,25.638831,CENTRO Y PLAZA COMERCIAL,GALERIAS VALLE ORIENTE,1144 Y 1146,Monterrey,NaN,42.2707,6098.849922,1578,12
2,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777,247,4
3,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922,1578,282
4,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922,1578,282


###2. Seleccionamos las tiendas candidatas

En la base de datos del estado de Nuevo León, se contaron con 4,000 empresas dedicadas a la venta de ropa, sin embargo, las Medianas y Grandes Empresas solo representaron el 2.1% del total. No obstante, este sector es altamente considerable que incluye tiendas como Bershka, Pull and Bearm Calvin Klein, Tt Blues Store, Zara, Sfera, American Eagle,  Forever 21, es decir, tiendas bien posicionadas que no consideran o no están especializadas en el mercado de las tallas extra. 

Es por ello que para la selección de tiendas se tomaron en cuenta todas las empresas de Tamaño Mediano y Grande. Para la selección de las tiendas de tamaño pequeño, se tuvieron que tener criterios de selección específicos, los cuales fueron: 

*   Porcentaje de Obesidad por municipio
*   Número por municipio 
*   Concentración de tiendas por colonia
*   Concentración de tiendas por municipio

La búsqueda de los puntos de interés se realizó con base en las variables, para ser consideradas las tiendas debían tener un valor de la variable superior a la media.

In [ ]:
 # Realizamos el filtrado de las tiendas, y de las 4,000 tiendas que teníamos incialmente, nos quedamos con 800 tiendas
 nuevo_leon_poi = nuevo_leon_calculo.loc[(nuevo_leon_calculo['dimension'] == 'A') | (nuevo_leon_calculo['dimension'] == 'B') | ((nuevo_leon_calculo['numero_tiendas_colonia'] > 77) & (nuevo_leon_calculo['numero_tiendas_municipio'] > 746) & (nuevo_leon_calculo['porcentaje_obesidad'] > 42) & (nuevo_leon_calculo['valor_comercial'] > 4612) )]

In [ ]:
# Eliminamos los registros repetidos repetidos
nuevo_leon_poi = nuevo_leon_poi.reset_index(drop=True)

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             873 non-null    object 
 1   razon_social              221 non-null    object 
 2   clase                     873 non-null    object 
 3   estrato                   873 non-null    object 
 4   tipo_vialidad             873 non-null    object 
 5   calle                     873 non-null    object 
 6   num_exterior              796 non-null    float64
 7   num_interior              279 non-null    float64
 8   colonia                   873 non-null    object 
 9   codigo_postal             873 non-null    int32  
 10  ubicacion                 873 non-null    object 
 11  telefono                  193 non-null    float64
 12  correo                    132 non-null    object 
 13  pagina_web                94 non-null     object 
 14  tipo      

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_poi.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial,numero_tiendas_municipio,numero_tiendas_colonia
0,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777,247,4
1,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922,1578,282
2,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922,1578,282
3,Melody,MILANO OPERADORA SA DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,FRAY SERVANDO TERESA DE MIER,240.0,0.0,CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.MILANO.COM,Fijo,-100.314189,25.668391,NaN,NaN,NaN,Monterrey,C,42.2707,6098.849922,1578,276
4,Moda casual,NaN,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,70.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",8.183439e+09,MODA121@GMAIL.COM,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,70 Y 121,Monterrey,C,42.2707,6098.849922,1578,282


---
## API LivePopulartimes
---

Esta API nos permite hacer un scraping de datos sobre Google Maps en cada una de las tiendas del dataframe final, accediendo a datos como rating, horarios, tiempo promedio de estancia, número de comentarios y placeid.
La gran ventaja de esta API en comparación del API de Google Places, es que no estamos limitados por el número de peticiones solicitadas, ya que para usar el API de Google se requiere de un access token que después de cierto número de peticiones comienza a realizar un cargo monetario.
Además, la gran ventaja de usar esta API LivePopulartimes es que nos regresa información mucho más completa en comparación del API de Google.

###1. Descargamos los paquetes necesarios para usar el API de LivePopulartimes

In [ ]:
# Para usar el API necesitamos clonar el repositorio
!git clone https://github.com/GrocerCheck/LivePopularTimes

Cloning into 'LivePopularTimes'...
remote: Enumerating objects: 968, done.
remote: Counting objects: 100% (968/968), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 968 (delta 690), reused 920 (delta 653), pack-reused 0
Receiving objects: 100% (968/968), 49.48 MiB | 31.55 MiB/s, done.
Resolving deltas: 100% (690/690), done.


In [ ]:
# Nos movemos a la carpeta indicada
cd LivePopularTimes/

/content/LivePopularTimes


In [ ]:
# Instalamos 
pip install .

Processing /content/LivePopularTimes
ERROR: Could not find a version that satisfies the requirement calendar (from LivePopularTimes==1.1) (from versions: none)
ERROR: No matching distribution found for calendar (from LivePopularTimes==1.1)


In [ ]:
# Imprtamos el API
import livepopulartimes

###2. Realizamos las peticiones al API LivePopularTimes

In [ ]:
# Ejemplo de petición: Para ver un ejemplo de los datos de respuesta de esta pétición quita el # de comentario de la siguiente línea y ejecutalo.
# reques_test = livepopulartimes.get_populartimes_by_address("(H-Mart Dunbar) 5557 Dunbar Street, Vancouver BC, Canada")

In [ ]:
# Definimos un DataFrame vacío
df_response = pd.DataFrame({})

Esta API hace uso de la dirección de los lugares para realizar la búsqueda, sin embargo, debido a que las direcciones de los lugares no están estandarizadas en Google Maps, es requerido realizar algunas combinaciones de Nombre de tienda, calle, numero, colonia municipio, código postal, tipo de establecimiento, razón social, con el fin de obtener la información de la tienda que realmente se está buscando.
Para esto se realizaron 6 diferentes opciones de búsqueda y se llamó el API para los más de 800 registros de tiendas candidatas que se tienen dentro del dataframe.

In [ ]:
# Tomamos de entrada el dataframe nuevo_leon_poi (poins of interest) y buscamos registro por registro en el API LivePopulartimes para ver datos como popular_times, rating, rating_n (numero de comentarios), currect_popularity datos que se obtienen con WEB SCRAPPING de Google MAPS
for index, row in nuevo_leon_poi.iterrows():
    cp = str(row['codigo_postal']) # Lo convierto a string porque es numérico
    num_ext = str(row['num_exterior']) # Lo convierto a string porque es numérico
    razon_soc = str(row['razon_social']) # Lo convierto a string
    nom_plaza = str(row['nom_corredor_industrial']) # Lo convierto a string

    text = row['nombre_tienda'] + ", " + cp + ", " + row['municipio'] + ", " + "Nuevo León"
    r = livepopulartimes.get_populartimes_by_address(text)
    df_response_temp = pd.json_normalize(r) # Normalizo la petición
    print("Busqueda 1")

    if df_response_temp.iloc[0]['name'] == None :
      text = row['nombre_tienda'] + ", " + row['calle'] + ", " + num_ext + ", " + row['municipio'] + ", " + "Nuevo León"
      r = livepopulartimes.get_populartimes_by_address(text)
      df_response_temp = pd.json_normalize(r) # Normalizo la petición
      print("Busqueda 2")

      if df_response_temp.iloc[0]['name'] == None :
        text = row['nombre_tienda'] + ", " + row['calle'] + ", " + num_ext + ", " + row['colonia'] + ", " + row['municipio']
        r = livepopulartimes.get_populartimes_by_address(text)
        df_response_temp = pd.json_normalize(r) # Normalizo la petición
        print("Busqueda 3")

        if df_response_temp.iloc[0]['name'] == None :
          text = row['nombre_tienda'] + ", " + row['colonia'] + ", " + row['municipio']
          r = livepopulartimes.get_populartimes_by_address(text)
          df_response_temp = pd.json_normalize(r) # Normalizo la petición
          print("Busqueda 4")
            
          if df_response_temp.iloc[0]['name'] == None :
            if razon_soc == "" :
              razon_soc = row['nombre_tienda']
            text = razon_soc + ", " + row['municipio'] + ", " + "Nuevo León"
            r = livepopulartimes.get_populartimes_by_address(text)
            df_response_temp = pd.json_normalize(r) # Normalizo la petición
            print("Busqueda 5")

            if df_response_temp.iloc[0]['name'] == None :
              if nom_plaza == "" :
                nom_plaza = cp
              text = row['nombre_tienda'] + ", " + nom_plaza
              r = livepopulartimes.get_populartimes_by_address(text)
              df_response_temp = pd.json_normalize(r) # Normalizo la petición
              print("Busqueda 6")

    print(f'Respuesta final: {index}, {r}')
    df_response = df_response.append(df_response_temp, ignore_index=True) # Agregar elelemento al dataframe

Busqueda 1
Busqueda 2
Busqueda 3
Busqueda 4
Busqueda 5
Respuesta final: 0, {'rating': 4.5, 'rating_n': 241, 'populartimes': [{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 48, 63, 69, 64, 55, 56, 66, 55, 24, 0, 0, 0]}, {'name': 'Tuesday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 46, 56, 63, 65, 52, 41, 53, 50, 20, 0, 0, 0]}, {'name': 'Wednesday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 39, 51, 53, 48, 45, 57, 68, 55, 26, 0, 0, 0]}, {'name': 'Thursday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 57, 55, 61, 67, 55, 52, 58, 43, 15, 0, 0, 0]}, {'name': 'Friday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 30, 42, 47, 46, 47, 56, 60, 47, 23, 0, 0, 0]}, {'name': 'Saturday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 50, 69, 72, 72, 85, 100, 91, 58, 25, 0, 0, 0]}, {'name': 'Sunday', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 26, 37, 51, 50, 44, 48, 46, 28, 10, 0, 0, 0]}], 'time_spent': [10, 10], 'name': 'Shasa Plaza Fiesta San Agustín', 'place_id': 'Ch

In [ ]:
# Unimos los dataframe
nuevo_leon_poi_time = nuevo_leon_poi.join(df_response)

In [ ]:
# Visualizamos la información sobre los campos
nuevo_leon_poi_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nombre_tienda             873 non-null    object 
 1   razon_social              221 non-null    object 
 2   clase                     873 non-null    object 
 3   estrato                   873 non-null    object 
 4   tipo_vialidad             873 non-null    object 
 5   calle                     873 non-null    object 
 6   num_exterior              796 non-null    float64
 7   num_interior              279 non-null    float64
 8   colonia                   873 non-null    object 
 9   codigo_postal             873 non-null    int32  
 10  ubicacion                 873 non-null    object 
 11  telefono                  193 non-null    float64
 12  correo                    132 non-null    object 
 13  pagina_web                94 non-null     object 
 14  tipo      

In [ ]:
# Eliminamos la columna que no nos sirven
nuevo_leon_poi_time = nuevo_leon_poi_time.drop(columns=['name', 'address', 'categories', 'place_types', 'coordinates.lat', 'coordinates.lng', ])

Podemos ver que se agregaron campos como el rating, el número de comentarios,el tiempo usual que suelen pasara las personas en las tiendas, y los horarios populares por horas y días de la semana.

In [ ]:
# Visualizamos el dataframe generado
nuevo_leon_poi_time.head()

,nombre_tienda,razon_social,clase,estrato,tipo_vialidad,calle,num_exterior,num_interior,colonia,codigo_postal,ubicacion,telefono,correo,pagina_web,tipo,longitud,latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local,municipio,dimension,porcentaje_obesidad,valor_comercial,numero_tiendas_municipio,numero_tiendas_colonia,rating,rating_n,populartimes,time_spent,place_id,current_popularity,popular_times
0,"Monterrey, san agustin",SERVICIOS SHASA S DE RL DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,Batallón San Patricio,1000.0,0.0,REAL DE SAN AGUSTIN,66260,"SAN PEDRO GARZA GARCÍA, San Pedro Garza García...",NaN,NaN,WWW.SHASA.COM,Fijo,-100.336835,25.648094,CENTRO Y PLAZA COMERCIAL,PLAZA FIESTA SAN AGUSTIN,SN,San Pedro Garza Garcia,A,35.1730,4634.679777,247,4,4.5,241.0,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[10, 10]",ChIJ6fE4SW2-YoYRarYYMb2uDIU,None,"[[7, [[6, 0, '', '', '6 AM'], [7, 0, '', '', '..."
1,Ropa dama y cosmeticos,10 ROPA DAMA Y COSMETICOS,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,10.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,10,Monterrey,NaN,42.2707,6098.849922,1578,282,NaN,NaN,NaN,NaN,None,None,None
2,Ropa dama,106 ROPA DAMA,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,0.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,106,Monterrey,NaN,42.2707,6098.849922,1578,282,NaN,NaN,NaN,NaN,None,None,None
3,Melody,MILANO OPERADORA SA DE CV,"Comercio al por menor de ropa, excepto de bebé...",11 a 30 personas,AVENIDA,FRAY SERVANDO TERESA DE MIER,240.0,0.0,CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",NaN,NaN,WWW.MILANO.COM,Fijo,-100.314189,25.668391,NaN,NaN,NaN,Monterrey,C,42.2707,6098.849922,1578,276,3.8,5.0,NaN,NaN,ChIJB8AL-L6AhYARhYT0HnzPqNg,None,None
4,Moda casual,NaN,"Comercio al por menor de ropa, excepto de bebé...",0 a 5 personas,CALLE,SERAFIN PEÑA,1010.0,70.0,MONTERREY CENTRO,64000,"MONTERREY, Monterrey, NUEVO LEÓN",8.183439e+09,MODA121@GMAIL.COM,NaN,Fijo,-100.326419,25.667601,"TIANGUIS, BAZAR O PULGA",PULGA RIO MONTERREY,70 Y 121,Monterrey,C,42.2707,6098.849922,1578,282,4.1,5515.0,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[15, 45]",ChIJRZ7FCCK-YoYRLz6AgoPG-Ws,None,"[[7, None, 1], [1, [[6, 0, '', '', '6 AM'], [7..."


Guardamos el Dataframe Final que nos servirá para realizar algunos análisis y gráficas sobre los datos.

**Te invitamos a continuar en la siguiente Notebook**

In [ ]:
# Guardamos CSV con todos los datos
nuevo_leon_poi_time.to_csv('nuevo_leon_poi_time.csv')

---
## ¿Cuáles son los siguientes Pasos? ... ¡Algoritmos Genéticos!
---

### **En donde estamos parados**

En este modulo nos centramos en generar un Dataset tomando diferentes bases de datos, combinándolas, llamando APIs, depurando datos.

Tal como lo mencionamos en un inicio, el objetivo de este proyecto es entrar al mercado de la venta de ropa, pero especializándonos en el sector de las tallas extra. Para esto se analizaron las tiendas actualmente establecidas, identificándolas como puntos de interés, y además visualizándolos como potenciales canales de distribución para la venta física o como lugares clave para la difusión de publicidad para la venta on line. 

---

### **¿Qué son los Algoritmos Genéticos?**

Sin embargo, ¿cómo saber cuál es la distribución más adecuada de los puntos de interés? aquí es en donde entra el machine learning, la inteligencia artificial y los algoritmos genéticos. Este tipo de algoritmos nos permiten obtener óptimos resultados a problemas que tienen billones de soluciones, ya que utilizan procesos biológicos en el software que permiten encontrar respuestas a los problemas que tienen largos espacios de búsquedas, generando continuamente soluciones y evaluando que tan bien las soluciones se adecuan a la salida deseada.

Cuando los problemas se resuelven con un algoritmo genético, en lugar de preguntar por soluciones específicas, se pueden proveer características o reglas que la solución debe de tener para ser aceptada. Por ejemplo, cuando se llena un camión de carga, se proveen un par de reglas como las siguientes: cargar las cosas grandes primero, distribuir el peso en ambos lados, poner las cosas ligeras arriba, intercalas las cosas con los espacios vacíos para que no se muevan. Existen más restricciones que se pueden agregar a la solución potencial, dependiendo del resultado que queramos obtener.

Es por ello que un aspecto fundamental para solucionar problemas usando algoritmos genéticos es que se debe de proveer una retroalimentación que ayude al motor de búsqueda a seleccionar la mejor de dos opciones. Esta retroalimentación es llamada fitness.

---

### **¿Cómo funcionan?**

Los algoritmos genéticos (AG) funcionan entre el conjunto de soluciones de un problema llamado fenotipo, y el conjunto de individuos de una población natural, codificando la información de cada solución en una cadena, generalmente binaria, llamada cromosoma.

Los símbolos que forman la cadena son llamados genes. Cuando la representación de los cromosomas se hace con cadenas de dígitos binarios se le conoce como genotipo. Los cromosomas evolucionan a través de iteraciones, llamadas generaciones. En cada generación, los cromosomas son evaluados usando alguna medida de aptitud. 

Las siguientes generaciones (nuevos cromosomas), son generadas aplicando los operadores genéticos repetidamente, siendo estos los operadores de selección, cruzamiento, mutación y reemplazo.

---

### **¿Lo podemos aplicar a nuestro proyecto?**

¡Por supuesto!

En este módulo encontramos las suficientes variables para construir una función de fitness sobre nuestra población (puntos de interés - tiendas de ropa).


Generaremos una población inicial constituida por un conjunto de cromosomas los cuales representan las posibles soluciones del problema (Inicialización) y cada uno de estos cromosomas les aplicaremos la función de fitness (Evaluación) para saber que tan buena es la solución.

**CROMOSOMA**

El cromosoma estará formado por un conjunto de genes donde cada uno de ellos representará un posible punto de interés (tienda)

[x0, x1, x2, x3, ... , xm]
donde xi es el punto de interés con el id i.

Posible solución [0, 0, 1, 0, 1]

Donde el 0 en la posición inicial indica que el punto de interés con id=0 no es considerado como tienda candidata en dicha solución, y el 1 en la tercera posición indica que el punto de interés con id=2 sí es considerado como tienda candidata en dicha solución.

**FUNCIÓN FITNESS**

En este punto se consideran los posibles valores a implementar en la función de fitness (que generamos en nuestro dataframe final):

*   Número de tiendas: Indicar como dato de entrada cuantas tiendas deseamos considerar
*   Porcentaje de Obesidad: Población, porcentaje de obesidad de la población de acuerdo a zonas municipales. 
*   Número de ventas: Entre mayor valor, mayor importancia.
*   Concentración de tiendas por Colonia y Municipio (Densidad demográfica)
*   Rating y comentarios por tienda: Entre mayor valor, mayor importancia.
*   Tiempo de estancia: Tiempo de permanencia en cada tienda. 
*   Radio de cobertura de una tienda: con el fin de evitar colocar tiendas muy cerca entre si.

La idea de la función de fitness es poder trabajar con un valor abstracto que llamamos "demanda estimada" y que consiste en determinar a partir de distintos factores la posible demanda en una determinada zona con la idea de cubrir la mayor parte de la "demanda estimada".

---

